In [1]:
import numpy as np
import tensorflow as tf
import gzip
import pickle
import sys
from ddm.run_split import SplitMnistGenerator
from ddm.alg.cla_models_multihead import MFVI_IBP_NN, Vanilla_NN
#from ddm.alg.vcl import run_vcl, run_vcl_ibp
from copy import deepcopy

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

# IBP network

In [2]:
hidden_size = [50,50]
batch_size = None
no_epochs = 100
single_head = False

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
batch_size=None
single_head=True
in_dim, out_dim = data_gen.get_dims()
x_testsets, y_testsets = [], []
for task_id in range(data_gen.max_iter):
    x_train, y_train, x_test, y_test = data_gen.next_task()
    x_testsets.append(x_test)
    y_testsets.append(y_test)

    # Set the readout head to train
    head = 0 if single_head else task_id
    bsize = x_train.shape[0] if (batch_size is None) else batch_size
    
    # Train network with maximum likelihood to initialize first model
    if task_id == 0:
        ml_model = Vanilla_NN(in_dim, hidden_size, out_dim, x_train.shape[0])
        ml_model.train(x_train, y_train, task_id, no_epochs, bsize)
        mf_weights = ml_model.get_weights()
        mf_variances = None
        mf_betas = None
        ml_model.close_session()

    # Train on non-coreset data
    mf_model = MFVI_IBP_NN(in_dim, hidden_size, out_dim, x_train.shape[0], prev_means=mf_weights, 
                           prev_log_variances=mf_variances, prev_betas=mf_betas)
    mf_model.train(x_train, y_train, head, no_epochs, bsize)
    mf_weights, mf_variances, mf_betas = mf_model.get_weights()

    # Incorporate coreset data and make prediction
#     x_coresets = []
#     y_coresets = []
#     acc = get_scores(mf_model, x_testsets, y_testsets, x_coresets, y_coresets, hidden_size, no_epochs, single_head, batch_size)
#     all_acc = concatenate_results(acc, all_acc)

    mf_model.close_session()

Epoch: 0001 cost= 0.709855616
Epoch: 0006 cost= 0.508833766
Epoch: 0011 cost= 0.344816089
Epoch: 0016 cost= 0.212866649
Epoch: 0021 cost= 0.115406297
Epoch: 0026 cost= 0.057102289
Epoch: 0031 cost= 0.029369019
Epoch: 0036 cost= 0.016914217
Epoch: 0041 cost= 0.011101806
Epoch: 0046 cost= 0.008233303
Epoch: 0051 cost= 0.006507133
Epoch: 0056 cost= 0.005332742
Epoch: 0061 cost= 0.004565125
Epoch: 0066 cost= 0.004022558
Epoch: 0071 cost= 0.003590583
Epoch: 0076 cost= 0.003259411
Epoch: 0081 cost= 0.002997679
Epoch: 0086 cost= 0.002782080
Epoch: 0091 cost= 0.002604916
Epoch: 0096 cost= 0.002452118
Optimization Finished!
layer: 0
act: (100, ?, 50)
layer: 1
act: (100, ?, 50)
final din: 50
final dout: 2
W_last: <unknown>
_weights: <unknown>
_biases: <unknown>
act: (100, ?, 50, 1)
pred: <unknown>
layer: 0
act: (10, ?, 50)
layer: 1
act: (10, ?, 50)
final din: 50
final dout: 2
W_last: <unknown>
_weights: <unknown>
_biases: <unknown>
act: (10, ?, 50, 1)
pred: <unknown>
z img: (10, 784, 50)
z biase

KeyboardInterrupt: 

# Split MNIST

In [ ]:
hidden_size = [256, 256]
batch_size = None
no_epochs = 100
single_head = False

# Run vanilla VCL
tf.set_random_seed(12)
np.random.seed(1)

coreset_size = 0
data_gen = SplitMnistGenerator()
vcl_result = run_vcl_ibp(hidden_size, no_epochs, data_gen, batch_size, single_head) # runs loop of tasks, trains/re-trains vcl, evaluates vcl
print(vcl_result)

# # Run random coreset VCL
# tf.reset_default_graph()
# tf.set_random_seed(12)
# np.random.seed(1)
#
# coreset_size = 40
# data_gen = SplitMnistGenerator()
# rand_vcl_result = vcl.run_vcl(hidden_size, no_epochs, data_gen,
#     coreset.rand_from_batch, coreset_size, batch_size, single_head)
# print(rand_vcl_result)
#
# # Run k-center coreset VCL
# tf.reset_default_graph()
# tf.set_random_seed(12)
# np.random.seed(1)
#
# data_gen = SplitMnistGenerator()
# kcen_vcl_result = vcl.run_vcl(hidden_size, no_epochs, data_gen,
#     coreset.k_center, coreset_size, batch_size, single_head)
# print(kcen_vcl_result)

# Plot average accuracy
vcl_avg = np.nanmean(vcl_result, 1)
# rand_vcl_avg = np.nanmean(rand_vcl_result, 1)
# kcen_vcl_avg = np.nanmean(kcen_vcl_result, 1)
plt.plot(vcl_avg)
plt.show()

In [ ]:
vcl_avg